<a href="https://colab.research.google.com/github/aamnaakhtar/chatbot/blob/main/iBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#path = "/content/drive/MyDrive/intents.json"
with open("/content/drive/MyDrive/chatbot.json") as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)
# print(training_sentences)
# print(training_labels)
# print(labels)
# print(responses)

# import pandas as pd
# df  = pd.read_json("/content/drive/MyDrive/intents.json")

# for intent in df['intents']:
#     for pattern in intent['patterns']:
#         training_sentences.append(pattern)
#         training_labels.append(intent['tag'])
#     responses.append(intent['responses'])
    
#     if intent['tag'] not in labels:
#         labels.append(intent['tag'])
        
# num_classes = len(labels)
# print(training_sentences)
# print(training_labels)
# print(labels)
# print(responses)

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                        

In [ ]:
epochs = 1000
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/1000
2/2 [==============================] - 1s 19ms/step - loss: 2.0794 - accuracy: 0.0909
Epoch 2/1000
2/2 [==============================] - 0s 6ms/step - loss: 2.0787 - accuracy: 0.2121
Epoch 3/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.0780 - accuracy: 0.2121
Epoch 4/1000
2/2 [==============================] - 0s 8ms/step - loss: 2.0773 - accuracy: 0.2121
Epoch 5/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.0766 - accuracy: 0.2121
Epoch 6/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.0759 - accuracy: 0.2121
Epoch 7/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.0751 - accuracy: 0.2121
Epoch 8/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.0744 - accuracy: 0.2121
Epoch 9/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.0738 - accuracy: 0.2121
Epoch 10/1000
2/2 [==============================] - 0s 6ms/step - loss: 2.0731 - accuracy: 0.2121
Epoch 11/1000
2/2 

In [ ]:
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [ ]:
pip install colorama

In [ ]:












import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("/content/drive/MyDrive/intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "Human: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "Bot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
Human: hey
Bot: Good to see you again
Human: how are you
Bot: I apologise for the inconvenience caused. We will reach out to you shortly
Human: could you help me
Bot: Tell me your problem to assist you
Human: i am bored, tell me something interesting
Bot: It's a pleasure helping you
Human: have a complaint
Bot: I apologise for the inconvenience caused. We will reach out to you shortly
Human: you are amazing
Bot: I am a chatbot
Human: what is your name
Bot: I was named iBot
Human: that's helpful
Bot: Mention not. It's my duty.
Human: thanks
Bot: Happy to help!
Human: quit
